In [18]:
import sys
import os
import pandas as pd
import numpy as np
import itertools
from sklearn.externals import joblib

sys.path.insert(1, '..')

In [3]:
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV, LassoCV
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

In [4]:
from xgboost import XGBClassifier

/Users/dkn22/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
data_dir = 'data/external/'
train_df = pd.read_csv(data_dir + 'train.csv')
test_df = pd.read_csv(data_dir + 'test.csv')
sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')

In [11]:
test_ids = test_df['id'].values
target_cols = [y for y in train_df.columns if y not in ('id', 'comment_text')]
targets = train_df[target_cols]

In [12]:
tfidf_corpus = joblib.load('data/processed/tfidf_corpus.pickle')

In [13]:
train_text = tfidf_corpus[:train_df.shape[0]]
test_text = tfidf_corpus[train_df.shape[0]:]

In [16]:
selected_features = []

for c in target_cols:
    y = targets[c][:]
    lasso = LogisticRegressionCV(penalty='l1', scoring='roc_auc', solver='liblinear', n_jobs=-1)
    lasso.fit(train_text, y)
    selector = SelectFromModel(lasso, prefit=True)
    
    selected = selector.get_support()
    
    selected_features.append(selected)
    print('Label %s done' %c)

Label toxic done


Label severe_toxic done


Label obscene done


Label threat done


Label insult done


Label identity_hate done


In [30]:
joblib.dump(selected_features, 'selected_features.pickle')

['selected_features.pickle']

In [31]:
sf = np.array(selected_features)

In [34]:
final_feat = sum(sf, 0) > 0

In [38]:
X_train = train_text[:, final_feat]

In [41]:
xgb = ClassifierChain(XGBClassifier(n_estimators=1000))

In [42]:
xgb.fit(X_train, targets)